In [3]:
import os
os.environ["HF_HOME"] = "E:/huggingface"

#import transformers
from datasets import load_dataset


dataset = load_dataset("squad_it")
DEVICE = "cuda:0"

In [2]:
from get_model import get_model
model_name = "llamantino"

model, tokenizer = get_model(model_name)
#model = model.to(DEVICE) #togliere se in 4 bit

bin c:\Users\Samuele\AppData\Local\Programs\Python\Python311\Lib\site-packages\bitsandbytes\libbitsandbytes_cuda121.dll


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

c:\Users\Samuele\AppData\Local\Programs\Python\Python311\Lib\site-packages\transformers\generation\configuration_utils.py:392: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
c:\Users\Samuele\AppData\Local\Programs\Python\Python311\Lib\site-packages\transformers\generation\configuration_utils.py:397: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(


In [22]:
from get_prompt import get_prompt
# del generate_prompt
generate_prompt, stop = get_prompt(model_name) #stop sembra useless

In [9]:
def build_question(context, question):
    return f"Dato il seguente testo:\n{context}\nRispondi brevemente a questa domanda:\n{question}"

def build_answer(answer):
    return f"Risposta breve: {answer}"

In [10]:
import random

def get_shots(dataset, n):
    conversation = []
    for i in range(n):
        elem = random.choice(dataset["train"])
        q_shot = build_question(elem["context"], elem["question"])
        conversation.append(dict(
            role="user",
            text=q_shot
        ))
        conversation.append(dict(
            role="assistant",
            text=build_answer(elem["answers"]["text"][0])
        ))
    return conversation

In [7]:
chat = [
  {"role": "user", "content": "SHOT 1" },
  {"role": "assistant", "content": "ANSWER 1"},
  {"role": "user", "content": "SHOT 2"},
  {"role": "assistant", "content": "ANSWER 2"},
  {"role": "user", "content": "SHOT 3"},
  {"role": "assistant", "content": "ANSWER 3"},
  {"role": "user", "content": "DOMANDA"},
]
 
print(tokenizer.apply_chat_template(chat, tokenize=False))


No chat template is defined for this tokenizer - using the default template for the LlamaTokenizerFast class. If the default is not appropriate for your model, please set `tokenizer.chat_template` to an appropriate template. See https://huggingface.co/docs/transformers/main/chat_templating for more information.



<s>[INST] SHOT 1 [/INST] ANSWER 1 </s><s>[INST] SHOT 2 [/INST] ANSWER 2 </s><s>[INST] SHOT 3 [/INST] ANSWER 3 </s><s>[INST] DOMANDA [/INST]


In [11]:
import torch

def forward_model(*, prompt, model, tokenizer):
    if "pad_token" not in tokenizer.special_tokens_map:
        tokenizer.pad_token = tokenizer.eos_token

    input_ids = tokenizer(
        prompt,
        return_tensors='pt',
        padding=True,
    ).input_ids
    with torch.no_grad():
        input_ids = input_ids.to(DEVICE)
        output_ids = model.generate(
            input_ids=input_ids,
            max_new_tokens=32,
            pad_token_id=tokenizer.eos_token_id,
            prompt_lookup_num_tokens=10,
            temperature = 1
        )
        # print(output_ids)

    ret = []
    for i in range(0, len(output_ids)):
        generated_text = tokenizer.decode(
            output_ids[i],
            skip_special_tokens=True
        )
        #print(generated_text[len(prompt[i]):])
        generated_text = generated_text[len(prompt[i]):] #perchè in output_ids c'è anche il prompt!

        # if "\n" in generated_text:
        #     generated_text = generated_text[:generated_text.index("\n")] non necessario
        
        ret.append(generated_text.strip())

    return ret

In [ ]:
import time

t0 = time.time()
out = forward_model(
    prompt=["ciao mamma come stai?"],
    model=model,
    tokenizer=tokenizer,
)
print(time.time() - t0)

print(out[0])

In [30]:
def generate_prompt(conversation, do_continue=False):
    # prompt = """[INST]<<SYS>>\nSei un assistente disponibile, rispettoso e onesto. Rispondi sempre nel modo piu' utile possibile, pur essendo sicuro. Le risposte non devono includere contenuti dannosi, non etici, razzisti, sessisti, tossici, pericolosi o illegali. Assicurati che le tue risposte siano socialmente imparziali e positive. Se una domanda non ha senso o non e' coerente con i fatti, spiegane il motivo invece di rispondere in modo non corretto. Se non conosci la risposta a una domanda, non condividere informazioni false.\n<</SYS>>\n\n"""
    assert conversation[-1]['role'] == "user"
    ST, ET = "<s>", "</s>" #Start/End Text
    SUM, EUM = "[INST]", "[/INST]" #Start/End User Message
    prompt = ""
    for message in conversation:
        if message['role'] == "user":
            prompt += f"{ST}{SUM}{message['text']}{EUM}"
        elif message['role'] == "assistant":
            prompt += f"{message['text']}{ET}"
        else:
            raise ValueError(f"Role not found,\nFounded {message['role']}")
    

    if do_continue:
        prompt += "\n" # non mi convince
    
    return prompt

In [32]:
### CELLA DI PROVA DEL NUOVO GET PROMPT DI SAIGA-ITA-7B!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
import uuid
from tqdm import tqdm

batch_size = 1

predicted_answers = []
theoretical_answers = []

ds = dataset["test"].shuffle(seed=42)

total = 0
for i in range(3):
    elem     = ds[i].copy()
    ids      = elem["id"]
    context  = elem["context"]
    question = elem["question"]
    answers  = elem["answers"]

    model_inputs = []
    chat = get_shots(dataset, 3) + [
                dict(
                    role="user",
                    text=build_question(context, question)
                )
            ]
    model_inputs.append(
        generate_prompt(
        # scrive tre esempi non come  prompt ma come cronologia di chat: role: user -> text: domanda, role: assistant -> text: risposta breve
            chat,
            do_continue=False
        ) + " " + build_answer("").strip()
    )

print(model_inputs[0])

<s>[INST]Dato il seguente testo:
Nell' estate del 1986, Queen ha fatto il loro tour finale con Freddie Mercury. Un tour sold-out a sostegno di A Kind of Magic, ancora una volta hanno assunto Spike Edney, portando a lui essere soprannominato il quinto membro non ufficiale. Il brano clou del Magic Tour è stato al Wembley Stadium di Londra e ha portato al doppio album live Queen at Wembley, uscito su CD e come DVD da concerto dal vivo, che è andato cinque volte al platino negli Stati Uniti e quattro volte al platino nel Regno Unito. Regina non poteva prenotare Wembley per una terza notte, ma hanno fatto giocare a Knebworth Park. Lo spettacolo si è esaurito nel giro di due ore e oltre 120.000 fan hanno riempito il parco per quella che era la performance finale della regina con Mercury. Queen ha iniziato il tour al Råsunda Stadium di Stoccolma, in Svezia, e durante il tour la band ha eseguito un concerto a Slane Castle, Irlanda, davanti ad un pubblico di 95.000 spettatori, che ha rotto il r

In [33]:
import uuid
from tqdm import tqdm

batch_size = 1

predicted_answers = []
theoretical_answers = []

ds = dataset["test"].shuffle(seed=42)

bar = tqdm(ds.iter(batch_size=batch_size), total=len(ds))
total = 0
for elem in bar:
    try:
        ids = elem["id"]
        context = elem["context"]
        question = elem["question"]
        answers = elem["answers"]

        model_inputs = []
        for c, q in zip(context, question):
            model_inputs.append(
                generate_prompt(
                    get_shots(dataset, 3) + [
                        dict(
                            role="user",
                            text=build_question(c, q)
                        )
                    ],
                    do_continue=True
                ) + " " + build_answer("").strip()
            )
        # print(model_inputs[0])
        # break

        model_outputs = forward_model(
            prompt=model_inputs[0],
            model=model,
            tokenizer=tokenizer,
        )
        model_outputs = model_outputs[0].split("Risposta breve:")[-1] if "Risposta breve:" in model_outputs[0] else _
        print(f"DOMANDA:\n{model_inputs[0]}\n\nRISPOSTA\n{model_outputs}\nSOLUZIONE\n{answers[0]['text']}\n")#[520:]

        # for model_output, id in zip(model_outputs, ids):
        #     predicted_answers.append(dict(
        #         id=id,
        #         prediction_text=model_output,
        #     ))
        predicted_answers.append(dict(
                id=ids[0],
                prediction_text=model_outputs,
            ))

        for ans, id in zip(answers, ids):
            theoretical_answers.append(dict(
                id=id,
                answers=ans,
            ))
        
    except Exception as e:
        print(e)
        continue

  0%|          | 0/7609 [00:00<?, ?it/s]c:\Users\Samuele\AppData\Local\Programs\Python\Python311\Lib\site-packages\transformers\generation\configuration_utils.py:397: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
  0%|          | 1/7609 [00:04<10:17:53,  4.87s/it]

DOMANDA:
<s>[INST]Dato il seguente testo:
La ricerca da allora rivela che l' auto-esame inizia precocemente nell' adolescenza, ma il raggiungimento dell' identità raramente avviene prima dei 18 anni. L' anno matricola del college influenza significativamente lo sviluppo dell' identità, ma può in realtà prolungare la moratoria psicosociale incoraggiando il riesame degli impegni precedenti e l' ulteriore esplorazione di possibilità alternative senza incoraggiare la risoluzione. Per la maggior parte, l' evidenza ha supportato le fasi di Erikson: ciascuna di esse è correlata ai tratti di personalità da lui originariamente previsti. Gli studi confermano anche l' impermanenza delle fasi; non esiste un punto finale nello sviluppo dell' identità.
Rispondi brevemente a questa domanda:
C' è un ultimo punto finale nello sviluppo dell' identità?[/INST]Risposta breve: no</s><s>[INST]Dato il seguente testo:
Nell' esercizio finanziario chiuso al 31 luglio 2013, Imperial ha registrato un utile netto t

  0%|          | 2/7609 [00:12<13:44:07,  6.50s/it]

DOMANDA:
<s>[INST]Dato il seguente testo:
Sia l' allevamento intensivo che quello all' aperto presentano preoccupazioni per il benessere degli animali. Nei sistemi intensivi, il cannibalismo, il beccaggio delle piume e lo sfiato possono essere comuni, con alcuni agricoltori che utilizzano la rifilatura del becco come misura preventiva. Le malattie possono anche essere comuni e diffondersi rapidamente attraverso il gregge. In sistemi estensivi, gli uccelli sono esposti a condizioni meteorologiche avverse e sono vulnerabili ai predatori e agli uccelli selvatici portatori di malattie. I sistemi del fienile sono stati trovati per avere il peggior benessere degli uccelli. Nel sud-est asiatico, la mancanza di controllo della malattia nell' allevamento all' aperto è stata associata a focolai di influenza aviaria.
Rispondi brevemente a questa domanda:
Quali dure misure usano gli allevatori per impedire che i polli danneggino se stessi o gli altri?[/INST]Risposta breve: alcuni agricoltori che u

  0%|          | 3/7609 [00:15<10:48:27,  5.12s/it]

...

DOMANDA:
<s>[INST]Dato il seguente testo:
L' eccezionale romanzo della cavalleria cavalleresca Tirant lo Blanc (1490) di Martorell mostra un passaggio dai valori medievali a quelli rinascimentali, cosa che si può vedere anche nell' opera di Metge. Il primo libro prodotto con il tipo mobile nella penisola iberica è stato stampato in catalano.
Rispondi brevemente a questa domanda:
Chi ha scritto Tirant lo Blanc?[/INST]Risposta breve: Martorell</s><s>[INST]Dato il seguente testo:
Nel 1970, il Presidente Nasser morì e fu succeduto da Anwar Sadat. Sadat ha trasferito la fedeltà dell' Egitto alla guerra fredda dall' Unione Sovietica agli Stati Uniti, espellendo i consiglieri sovietici nel 1972. Ha lanciato la politica di riforma economica dell' Infitah, bloccando nel contempo l' opposizione religiosa e laica. Nel 1973, l' Egitto, insieme alla Siria, lanciò la guerra di ottobre, un attentato a sorpresa per riconquistare parte del territorio del Sinai che Israele aveva catturato 6 anni prima. 

  0%|          | 8/7609 [00:35<9:20:09,  4.42s/it]

DOMANDA:
<s>[INST]Dato il seguente testo:
A partire dal 2015[aggiornamento], la Nigeria è la ventesima più grande economia mondiale, con un valore rispettivamente di oltre 500 miliardi di dollari e 1 trilione di dollari in termini di PIL nominale e di parità del potere d' acquisto. Ha superato il Sudafrica per diventare la più grande economia africana nel 2014. Inoltre, il rapporto debito/PIL è solo dell' 11 per cento, ossia l' 8 per cento al di sotto del rapporto 2012. La Nigeria è considerata un mercato emergente dalla Banca Mondiale; è stata identificata come una potenza regionale del continente africano, una potenza centrale negli affari internazionali ed è stata anche identificata come una potenza globale emergente. La Nigeria è membro del gruppo di paesi MINT, che sono ampiamente visti come le prossime economie "simili ai BRIC" del mondo. E' anche elencato tra le "prossime undici" economie destinate a diventare tra le più grandi del mondo. La Nigeria è membro fondatore del Common

  0%|          | 8/7609 [00:39<10:26:14,  4.94s/it]


KeyboardInterrupt: 

In [ ]:
print(model_inputs[0])

In [ ]:
model_outputs

In [ ]:
predicted_answers[-1]
#theoretical_answers[0]

In [34]:
import os
import json


# Create the directory if it doesn't exist
os.makedirs('./cache', exist_ok=True)
with open(f"./cache/generated-squad-{model_name}.json", "w") as f:
    json.dump(dict(
        predicted_answers=predicted_answers,
        theoretical_answers=theoretical_answers,
    ), f, indent=4)

In [35]:
import json
with open(f"./cache/generated-squad-{model_name}.json", "r") as f:
    data = json.load(f)

print(len(data["predicted_answers"]))

8


In [36]:
import evaluate

predicted_answers = data["predicted_answers"]
theoretical_answers = data["theoretical_answers"]

metric = evaluate.load("squad")
results = metric.compute(predictions=predicted_answers, references=theoretical_answers)

import time


print("=== REPORT ===")
print("current date:", time.strftime("%d/%m/%Y %H:%M:%S"))
print("Dataset: SQuAD-it")
print("Model:", model_name)
print(results)
print("==========================")

=== REPORT ===
current date: 24/01/2024 12:40:00
Dataset: SQuAD-it
Model: llamantino
{'exact_match': 37.5, 'f1': 58.698534255599476}
